<a href="https://colab.research.google.com/github/kplam624/Mask-Finder/blob/imagepreprocessing/image_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import dependencies
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np

import PIL

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import (
    VGG19,
    preprocess_input,
    decode_predictions
)

In [2]:
names = ['with_mask', 'without_mask']
batch = 32

# Code if run on Google Colab

In [3]:
# When Running on google colab use the following to install the split-folders library
!pip install split-folders 
import splitfolders

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
Data = '/content/drive/MyDrive/Colab Notebooks/maskfinder/Data'

In [6]:
# Line below will split the folders into test, validation, and test data.
# Only run when first using the notebook!!

splitfolders.ratio(Data, output="output", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 11043 files [44:19,  4.15 files/s]


In [7]:
# Define what the train, test, and validation datasets are.
train_data = '/content/output/train'
test_data = '/content/output/test'
val_data = '/content/output/val'

# If Running on a jupyter notebook run this

In [ ]:
import splitfolders

In [ ]:
# Only run the code below if you are running the code for the first time.
# Data = 'Data'
# splitfolders.ratio(Data, output = "output", seed = 1337, ratio=(.8, 0.1, 0.1))

In [ ]:
# Define what the train, test, and validation datasets are.
train_data = 'output/train'
test_data = 'output/test'
val_data = 'output/val'

## The training is the same whether for colab or for Jupyter notebook

In [20]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
train_datagen

In [9]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [21]:
train_generator = train_datagen.flow_from_directory(
    directory = train_data,
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical')

Found 8833 images belonging to 2 classes.


In [22]:
test_generator = test_datagen.flow_from_directory(
    directory = test_data,
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical')

Found 1106 images belonging to 2 classes.


In [25]:
val_generator = test_datagen.flow_from_directory(
    directory = val_data,
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical')

Found 1104 images belonging to 2 classes.


In [12]:
from keras import Sequential
from keras.layers import Flatten, Dense

In [13]:
vgg19 = VGG19(weights = "imagenet", include_top = False, input_shape = (220,220,3))

for layers in vgg19.layers:
    layers.trainable = False

80142336/80134624 [==============================] - 1s 0us/step


In [27]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(units = 2, activation = 'sigmoid'))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 36866     
Total params: 20,061,250
Trainable params: 36,866
Non-trainable params: 20,024,384
_________________________________________________________________


In [29]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [30]:
# This is to further train the model.
# Keep in mind that to further train the model, a strong processor/ computing power is needed.
# A common error that occurs is 'UnidentifiedImageError'
# Try in google colab and if the error still occurs, you do not have the processing power for the size of the data.

model.fit(train_generator,
            steps_per_epoch = len(train_generator)//32,
            epochs = 15,
            validation_data = val_generator,
            validation_steps=len(val_generator)//32,
            verbose = 2
            )

Epoch 1/15
8/8 - 176s - loss: 0.9780 - accuracy: 0.6328 - val_loss: 0.4042 - val_accuracy: 0.8750
Epoch 2/15
8/8 - 176s - loss: 0.4159 - accuracy: 0.7969 - val_loss: 0.0982 - val_accuracy: 0.9375
Epoch 3/15
8/8 - 177s - loss: 0.2257 - accuracy: 0.9258 - val_loss: 0.3960 - val_accuracy: 0.9062
Epoch 4/15
8/8 - 177s - loss: 0.2198 - accuracy: 0.9297 - val_loss: 0.0842 - val_accuracy: 0.9375
Epoch 5/15
8/8 - 176s - loss: 0.1015 - accuracy: 0.9648 - val_loss: 0.1924 - val_accuracy: 0.9688
Epoch 6/15
8/8 - 174s - loss: 0.0845 - accuracy: 0.9805 - val_loss: 0.1695 - val_accuracy: 0.9688
Epoch 7/15
8/8 - 173s - loss: 0.1076 - accuracy: 0.9609 - val_loss: 0.1562 - val_accuracy: 0.9375
Epoch 8/15
8/8 - 175s - loss: 0.0681 - accuracy: 0.9805 - val_loss: 0.0917 - val_accuracy: 0.9688
Epoch 9/15
8/8 - 177s - loss: 0.0883 - accuracy: 0.9609 - val_loss: 0.0418 - val_accuracy: 1.0000
Epoch 10/15
8/8 - 156s - loss: 0.0623 - accuracy: 0.9733 - val_loss: 0.1834 - val_accuracy: 0.9375
Epoch 11/15
8/8 - 1

In [31]:
model.evaluate(test_generator)

35/35 [==============================] - 644s 18s/step - loss: 0.0800 - accuracy: 0.9747


[0.08002479374408722, 0.9746835231781006]

In [33]:
model.save('face.h5')